## This notebook performs a single science visit for this run for a target HD 
### This should be run AFTER performing focus on nearby star HD 185975
##### The script, slews and acquires the target, applies focus correction, takes sequence of data

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-29061**

In [ ]:
TARGET="HD115169" #"HD105070" # 

In [ ]:
import os
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control.utils import RotType
from astropy import time as astropytime

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
# make sure all remotes etc are running
script = LatissAcquireAndTakeSequence(index=os.getuid())  # this essentially calls the init method of the script
#await asyncio.sleep(10) # This can be removed in the future...
await script.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
# script.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

In [ ]:
# Do acquisition in the same mode we'll use to focus afterwards

### Acquire using the hologram grating
##### This is the same position as the ronchi170lpmm grating

In [ ]:
acq2_yaml = yaml.safe_dump({"object_name": TARGET,
                            "do_acquire": True,
                            "do_take_sequence": False,
                            "acq_filter" : 'RG610',
                            "acq_grating" : 'holo4_003', 
                            "acq_exposure_time": 0.5,
                            "max_acq_iter": 4,
                            "target_pointing_tolerance": 2,
                            "do_pointing_model": False,
                            "dataPath": '/project/shared/auxTel/rerun/quickLook',
                            "target_pointing_verification": False})
print(acq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
acq2_configuration = script.cmd_configure.DataType()
acq2_configuration.config = acq2_yaml
await script.do_configure(acq2_configuration)

In [ ]:
# This take the acquisition sequence for the hologram (and Ronchi170lpmm grating)
acq2_results = await script.arun()

In [ ]:
# await script.atcs.stop_tracking()

In [ ]:
# await script.atcs.point_azel(az=265, el=75)

In [ ]:
# script above defaults to paralactic angle = 0, but normally we use it at 90
# look at offsets from previous acquisition and add them here
# 29.9, 177.7
await script.atcs.slew_object(TARGET, offset_x=29.9, offset_y=177.7, rot_type=RotType.Parallactic, rot=0)

In [ ]:
await script.latiss.take_object(exptime=2., grating='holo4_003', filter='empty_1')

In [ ]:
target = np.array((1750, 300))
curr = np.array((1770,135))

In [ ]:
(target-curr)*0.1

In [ ]:
await script.atcs.offset_xy(x=2,y=-16.5)

In [ ]:
dx=60*2 # +/- offset range in x [arcsec]
xstep_size=45 # +/- step size in x [arcsec]
xoffset=np.arange(-dx,dx+1,xstep_size)
print(xoffset)

In [ ]:
ystep_size=50 # +/- step size in x [arcsec]
# go from -15 arcsec to ~
yoffset=np.arange(-15,60*5,ystep_size)
yoffset=np.arange(-60*4, 25, ystep_size)
print(yoffset)

In [ ]:
# Only run this when target is on the [X,Y] pixel of 1750, 300 (middle left, bottom of detector)
group_id = astropytime.Time.now().tai.isot
for i, yoff in enumerate(yoffset):
    for j, xoff in enumerate(xoffset):
        print(f'yoff is {yoff}, xoff is {xoff}')
        # offset the telescope, using ABSOLUTE offsets
        await script.atcs.offset_xy(x=xoff, y=yoff, relative=False)
        # take 20s image
        await script.latiss.take_object(exptime=20, n=1, group_id=group_id, grating='holo4_003', filter='empty_1')

In [ ]:
dx=14 # +/- offset range in x [arcsec]
xstep_size=5 # +/- step size in x [arcsec]
xoffset=np.arange(-dx,dx+1,xstep_size)
print(xoffset)

In [ ]:
ystep_size=8 # +/- step size in x [arcsec]
# go from -15 arcsec to ~
# yoffset=np.arange(-15,65,ystep_size) # backwards
yoffset=np.arange(-35,10,ystep_size)
print(yoffset)

In [ ]:
# Only run this when target is on the [X,Y] pixel of 1750, 300 (middle left, bottom of detector)
group_id = astropytime.Time.now().tai.isot
for i, yoff in enumerate(yoffset):
    for j, xoff in enumerate(xoffset):
        print(f'yoff is {yoff}, xoff is {xoff}')
        # offset the telescope, using ABSOLUTE offsets
        await script.atcs.offset_xy(x=xoff, y=yoff, relative=False)
        # take 20s image
        await script.latiss.take_object(exptime=20, n=1, group_id=group_id)

In [ ]:
yoff

In [ ]:
xoff

In [ ]:
ystep_size=8 # +/- step size in x [arcsec]
# go from -15 arcsec to ~
# yoffset=np.arange(-15,65,ystep_size) # backwards
yoffset=np.arange(10,32,ystep_size)
print(yoffset)

In [ ]:
# Only run this when target is on the [X,Y] pixel of 1750, 300 (middle left, bottom of detector)
group_id = astropytime.Time.now().tai.isot
for i, yoff in enumerate(yoffset):
    for j, xoff in enumerate(xoffset):
        print(f'yoff is {yoff}, xoff is {xoff}')
        # offset the telescope, using ABSOLUTE offsets
        await script.atcs.offset_xy(x=xoff, y=yoff, relative=False)
        # take 20s image
        await script.latiss.take_object(exptime=20, n=1, group_id=group_id)

### Now do test 2

In [ ]:
# First Acquire
TARGET="HD160617" 

In [ ]:
acq2_yaml = yaml.safe_dump({"object_name": TARGET,
                            "do_acquire": True,
                            "do_take_sequence": False,
                            "acq_filter" : 'empty_1',
                            "acq_grating" : 'holo4_003', 
                            "acq_exposure_time": 1.0,
                            "max_acq_iter": 4,
                            "target_pointing_tolerance": 2,
                            "do_pointing_model": False,
                            "dataPath": '/project/shared/auxTel/rerun/quickLook',
                            "target_pointing_verification": False})
print(acq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
acq2_configuration = script.cmd_configure.DataType()
acq2_configuration.config = acq2_yaml
await script.do_configure(acq2_configuration)

In [ ]:
# This take the acquisition sequence for the hologram (and Ronchi170lpmm grating)
acq2_results = await script.arun()

In [ ]:
seq2_yaml = yaml.safe_dump({"object_name": TARGET,
                                "do_acquire": False,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [20, 20, 20,
                                                            20, 20, 20,
                                                            20, 20, 20], 
                                "filter_sequence": ['RG610','RG610', 'RG610',
                                                    'BG40','BG40', 'BG40',
                                                    'empty_1','empty_1', 'empty_1'], 
                                # RG610 and Ronchi
                                "grating_sequence": ['holo4_003', 'holo4_003', 'holo4_003',
                                                     'holo4_003', 'holo4_003', 'holo4_003',
                                                     'holo4_003', 'holo4_003', 'holo4_003'],
                                "do_pointing_model": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq2_configuration = script.cmd_configure.DataType()
seq2_configuration.config = seq2_yaml
await script.do_configure(seq2_configuration)

In [ ]:
# This take the sequence of images with the Ronchi
seq2_results = await script.arun()

# Test3: Now to PNE

In [ ]:
# First Acquire
target_ra="17:21:04"
target_dec="-29:02:59"

In [ ]:
await script.atcs.slew_icrs(ra=target_ra, dec=target_dec, offset_x=29.9, offset_y=177.7, rot_type=RotType.Parallactic, rot=90)

In [ ]:
group_id = astropytime.Time.now().tai.isot
await script.latiss.take_object(exptime=60, n=1, group_id=group_id, grating='empty_1', filter='empty_1')

In [ ]:
seq2_yaml = yaml.safe_dump({"object_ra": target_ra, "object_dec": target_dec,
                                "do_acquire": False,
                                "do_take_sequence": True,
                                "exposure_time_sequence" : [60, 60, 60,
                                                            60, 60, 60,
                                                            60, 60, 60], 
                                "filter_sequence": ['empty_1','empty_1', 'empty_1',
                                                    'empty_1','empty_1', 'empty_1',
                                                    'empty_1','empty_1', 'empty_1'], 
                                # RG610 and Ronchi
                                "grating_sequence": ['holo4_003', 'holo4_003', 'holo4_003',
                                                     'holo4_003', 'holo4_003', 'holo4_003',
                                                     'holo4_003', 'holo4_003', 'holo4_003'],
                                "do_pointing_model": False,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook',
                                "target_pointing_verification": False})
print(seq2_yaml)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
seq2_configuration = script.cmd_configure.DataType()
seq2_configuration.config = seq2_yaml
await script.do_configure(seq2_configuration)

In [ ]:
# This take the sequence of images with the Ronchi
seq2_results = await script.arun()